In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Thyroid_Cancer"
cohort = "GSE58689"

# Input paths
in_trait_dir = "../../input/GEO/Thyroid_Cancer"
in_cohort_dir = "../../input/GEO/Thyroid_Cancer/GSE58689"

# Output paths
out_data_file = "../../output/preprocess/Thyroid_Cancer/GSE58689.csv"
out_gene_data_file = "../../output/preprocess/Thyroid_Cancer/gene_data/GSE58689.csv"
out_clinical_data_file = "../../output/preprocess/Thyroid_Cancer/clinical_data/GSE58689.csv"
json_path = "../../output/preprocess/Thyroid_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression signature associated with BRAFV600E mutation in human papillary thyroid carcinoma based on transgenic mouse model"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['type of thyroid tissue: papillary thyroid carcinoma', 'type of thyroid tissue: asymptomatic thyroid', 'type of thyroid tissue: borderline lesion', 'type of thyroid tissue: benign hyperplastic lesion'], 1: ['brafv600e mutation: positive', 'brafv600e mutation: not present'], 2: ['Sex: female', 'Sex: male'], 3: ['age (months): 21', 'age (months): 10', 'age (months): 12', 'age (months): 11', 'age (months): 13', 'age (months): 7', 'age (months): 6', 'age (months): 24', 'age (months): 4']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability Analysis
# Based on the Series title "Gene expression signature...", this dataset likely contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait: Looking at key 0, we can see "type of thyroid tissue" which indicates normal vs cancer samples
trait_row = 0

# For gender: Looking at key 1, we can see "Sex: female" and "Sex: male"
# Also in key 2 we see "Sex: female" and "Sex: male"
gender_row = 1  # Using the first occurrence

# For age: Looking at key 2, we see "age (years): XX" for multiple entries
age_row = 2

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """Convert thyroid tissue type to binary: 1 for cancer, 0 for normal."""
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip().lower()
        if 'papillary thyroid carcinoma' in value or 'cancer' in value:
            return 1
        elif 'normal' in value:
            return 0
    return None

def convert_age(value):
    """Convert age to continuous numeric value."""
    if isinstance(value, str) and ':' in value:
        try:
            return float(value.split(':', 1)[1].strip())
        except (ValueError, TypeError):
            pass
    return None

def convert_gender(value):
    """Convert gender to binary: 0 for female, 1 for male."""
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip().lower()
        if 'female' in value:
            return 0
        elif 'male' in value:
            return 1
    return None

# 3. Save Metadata - Initial filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical data
    print("Preview of selected clinical features:")
    print(preview_df(selected_clinical_df))
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{'GSM1413270': [1.0, nan, nan], 'GSM1413271': [nan, nan, nan], 'GSM1413272': [nan, nan, nan], 'GSM1413273': [1.0, nan, nan], 'GSM1413274': [1.0, nan, nan], 'GSM1413275': [1.0, nan, nan], 'GSM1413276': [1.0, nan, nan], 'GSM1413277': [nan, nan, nan], 'GSM1413278': [1.0, nan, nan], 'GSM1413279': [nan, nan, nan], 'GSM1413280': [1.0, nan, nan], 'GSM1413281': [nan, nan, nan], 'GSM1413282': [nan, nan, nan], 'GSM1413283': [nan, nan, nan], 'GSM1413284': [nan, nan, nan], 'GSM1413285': [nan, nan, nan], 'GSM1413286': [nan, nan, nan], 'GSM1413287': [nan, nan, nan], 'GSM1413288': [1.0, nan, nan], 'GSM1413289': [1.0, nan, nan], 'GSM1413290': [nan, nan, nan], 'GSM1413291': [nan, nan, nan], 'GSM1413292': [nan, nan, nan], 'GSM1413293': [nan, nan, nan], 'GSM1413294': [nan, nan, nan], 'GSM1413295': [nan, nan, nan], 'GSM1413296': [1.0, nan, nan], 'GSM1413297': [nan, nan, nan], 'GSM1413298': [nan, nan, nan], 'GSM1413299': [nan, nan, nan], 'GSM1413300': [nan, nan, nan],

### Step 3: Gene Data Extraction

In [4]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Initially assume gene data is available

# First check if the matrix file contains the expected marker
found_marker = False
try:
    with gzip.open(matrix_file, 'rt') as file:
        for line in file:
            if "!series_matrix_table_begin" in line:
                found_marker = True
                break
    
    if found_marker:
        print("Found the matrix table marker in the file.")
    else:
        print("Warning: Could not find '!series_matrix_table_begin' marker in the file.")
        
    # Try to extract gene data from the matrix file
    gene_data = get_genetic_data(matrix_file)
    
    if gene_data.shape[0] == 0:
        print("Warning: Extracted gene data has 0 rows.")
        is_gene_available = False
    else:
        print(f"Gene data shape: {gene_data.shape}")
        # Print the first 20 gene/probe identifiers
        print("First 20 gene/probe identifiers:")
        print(gene_data.index[:20].tolist())
        
except Exception as e:
    print(f"Error extracting gene data: {e}")
    is_gene_available = False
    
    # Try to diagnose the file format
    print("Examining file content to diagnose the issue:")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            for i, line in enumerate(file):
                if i < 10:  # Print first 10 lines to diagnose
                    print(f"Line {i}: {line.strip()[:100]}...")  # Print first 100 chars of each line
                else:
                    break
    except Exception as e2:
        print(f"Error examining file: {e2}")

if not is_gene_available:
    print("Gene expression data could not be successfully extracted from this dataset.")


SOFT file: ../../input/GEO/Thyroid_Cancer/GSE58689/GSE58689_family.soft.gz
Matrix file: ../../input/GEO/Thyroid_Cancer/GSE58689/GSE58689-GPL6246_series_matrix.txt.gz
Found the matrix table marker in the file.
Gene data shape: (35556, 38)
First 20 gene/probe identifiers:
['10338001', '10338002', '10338003', '10338004', '10338005', '10338006', '10338007', '10338008', '10338009', '10338010', '10338011', '10338012', '10338013', '10338014', '10338015', '10338016', '10338017', '10338018', '10338019', '10338020']


### Step 4: Gene Identifier Review

In [5]:
# These identifiers are not standard human gene symbols
# They appear to be Affymetrix probe IDs from the GPL96 platform (HG-U133A)
# These need to be mapped to official human gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Get a more complete view to understand the annotation structure
print("\nComplete sample of a few rows:")
print(gene_annotation.iloc[:3].to_string())

# Check for columns that might contain gene information
potential_gene_columns = [col for col in gene_annotation.columns if 
                          any(term in col.upper() for term in ["GENE", "SYMBOL", "NAME", "ID"])]
print(f"\nPotential gene-related columns: {potential_gene_columns}")

# Correctly identify the columns for probe ID and gene symbols
gene_id_col = 'ID'  # This is the probe identifier column
gene_symbol_col = 'Gene Symbol'  # Corrected to match the actual column name in the data

# Verify columns exist before mapping
if gene_id_col in gene_annotation.columns and gene_symbol_col in gene_annotation.columns:
    # Create the mapping using the library function
    mapping_data = get_gene_mapping(gene_annotation, gene_id_col, gene_symbol_col)
    
    # Print sample of the mapping to confirm
    print("\nSample of probe ID to gene symbol mappings:")
    print(mapping_data.head(10))
    
    # Check the size of the mapping data
    print(f"\nTotal number of probe-to-gene mappings: {len(mapping_data)}")
else:
    missing_cols = []
    if gene_id_col not in gene_annotation.columns:
        missing_cols.append(gene_id_col)
    if gene_symbol_col not in gene_annotation.columns:
        missing_cols.append(gene_symbol_col)
    print(f"\nError: The following columns are missing from the annotation data: {missing_cols}")



Gene annotation preview:
Columns in gene annotation: ['ID', 'GB_ACC', 'SPOT_ID', 'Species Scientific Name', 'Annotation Date', 'Sequence Type', 'Sequence Source', 'Target Description', 'Representative Public ID', 'Gene Title', 'Gene Symbol', 'ENTREZ_GENE_ID', 'RefSeq Transcript ID', 'Gene Ontology Biological Process', 'Gene Ontology Cellular Component', 'Gene Ontology Molecular Function']
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 

### Step 6: Gene Identifier Mapping

In [7]:
# STEP 1: Extract relevant gene data and mapping information
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Get gene expression data
gene_expression_df = get_genetic_data(matrix_file)

# Get gene annotation data
gene_annotation = get_gene_annotation(soft_file)

# STEP 2: Create gene mapping dataframe
# Based on the preview from the previous step, we know:
# - 'ID' in gene_annotation corresponds to probe identifiers (matches the gene expression index)
# - 'Gene Symbol' in gene_annotation contains the gene symbols we want to map to
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')

# STEP 3: Apply the gene mapping to convert probe-level to gene-level expression
gene_data = apply_gene_mapping(gene_expression_df, mapping_df)

# Preview the mapped gene data
print("\nGene data shape after mapping (rows x columns):", gene_data.shape)
print("First 10 gene symbols:", gene_data.index[:10].tolist())
print("Number of unique genes:", len(gene_data.index))



Gene data shape after mapping (rows x columns): (75889, 38)
First 10 gene symbols: ['A-', 'A-1', 'A-2-', 'A-211C6', 'A-3', 'A-4', 'A-52', 'A-BETA', 'A-C1', 'A-D']
Number of unique genes: 75889


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
try:
    # Make sure the directory exists
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    
    # Use the gene_data variable from the previous step (don't try to load it from file)
    print(f"Gene data shape before normalization: {gene_data.shape}")
    
    # Apply normalization to gene symbols
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    
    # Save the normalized gene data
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
    
    # Use the normalized data for further processing
    gene_data = normalized_gene_data
    is_gene_available = True
except Exception as e:
    print(f"Error normalizing gene data: {e}")
    is_gene_available = False

# 2. Load clinical data with correct row indices
try:
    # Load the clinical data from file
    soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    background_info, clinical_data = get_background_and_clinical_data(matrix_file)
    
    # Debug the trait and gender conversion functions
    print("Sample conversion results:")
    sample_trait = list(clinical_data.iloc[0])[1]  # Get a sample from trait row
    sample_gender = list(clinical_data.iloc[1])[1]  # Get a sample from gender row
    print(f"Sample trait value: '{sample_trait}' -> {convert_trait(sample_trait)}")
    print(f"Sample gender value: '{sample_gender}' -> {convert_gender(sample_gender)}")
    
    # Extract clinical features with CORRECT indices from Step 2
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=0,       # Correct trait row from Step 2
        convert_trait=convert_trait,
        gender_row=1,      # Correct gender row from Step 2
        convert_gender=convert_gender,
        age_row=None,      # Age row is None as per Step 2
        convert_age=None
    )
    
    print(f"Extracted clinical data shape: {clinical_features.shape}")
    print("Preview of clinical data (first 5 samples):")
    print(clinical_features.iloc[:, :5])
    
    # Save the properly extracted clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    
    is_trait_available = True
except Exception as e:
    print(f"Error extracting clinical data: {e}")
    is_trait_available = False

# 3. Link clinical and genetic data if both are available
if is_trait_available and is_gene_available:
    try:
        # Debug the column names to ensure they match
        print(f"Gene data columns (first 5): {gene_data.columns[:5].tolist()}")
        print(f"Clinical data columns (first 5): {clinical_features.columns[:5].tolist()}")
        
        # Check for common sample IDs
        common_samples = set(gene_data.columns).intersection(clinical_features.columns)
        print(f"Found {len(common_samples)} common samples between gene and clinical data")
        
        if len(common_samples) > 0:
            # Link the clinical and genetic data
            linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
            print(f"Initial linked data shape: {linked_data.shape}")
            
            # Debug the trait values before handling missing values
            print("Preview of linked data (first 5 rows, first 5 columns):")
            print(linked_data.iloc[:5, :5])
            
            # Handle missing values
            linked_data = handle_missing_values(linked_data, trait)
            print(f"Linked data shape after handling missing values: {linked_data.shape}")
            
            if linked_data.shape[0] > 0:
                # Check for bias in trait and demographic features
                is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
                
                # Validate the data quality and save cohort info
                note = "Dataset contains gene expression data from thyroid cancer samples with tumor type information."
                is_usable = validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=is_biased,
                    df=linked_data,
                    note=note
                )
                
                # Save the linked data if it's usable
                if is_usable:
                    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                    linked_data.to_csv(out_data_file)
                    print(f"Linked data saved to {out_data_file}")
                else:
                    print("Data not usable for the trait study - not saving final linked data.")
            else:
                print("After handling missing values, no samples remain.")
                validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=True,
                    df=pd.DataFrame(),
                    note="No valid samples after handling missing values."
                )
        else:
            print("No common samples found between gene expression and clinical data.")
            validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=is_gene_available,
                is_trait_available=is_trait_available,
                is_biased=True,
                df=pd.DataFrame(),
                note="No common samples between gene expression and clinical data."
            )
    except Exception as e:
        print(f"Error linking or processing data: {e}")
        validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available,
            is_biased=True,  # Assume biased if there's an error
            df=pd.DataFrame(),  # Empty dataframe for metadata
            note=f"Error in data processing: {str(e)}"
        )
else:
    # We can't proceed with linking if either trait or gene data is missing
    print("Cannot proceed with data linking due to missing trait or gene data.")
    validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available,
        is_biased=True,  # Data is unusable if we're missing components
        df=pd.DataFrame(),  # Empty dataframe for metadata
        note="Missing essential data components for linking (trait data or gene expression data)."
    )

Gene data shape before normalization: (75889, 38)
Gene data shape after normalization: (3836, 38)


Normalized gene data saved to ../../output/preprocess/Thyroid_Cancer/gene_data/GSE58689.csv


Sample conversion results:
Sample trait value: 'type of thyroid tissue: papillary thyroid carcinoma' -> 1
Sample gender value: 'brafv600e mutation: positive' -> None
Extracted clinical data shape: (2, 38)
Preview of clinical data (first 5 samples):
                GSM1413270  GSM1413271  GSM1413272  GSM1413273  GSM1413274
Thyroid_Cancer         1.0         NaN         NaN         1.0         1.0
Gender                 NaN         NaN         NaN         NaN         NaN
Clinical data saved to ../../output/preprocess/Thyroid_Cancer/clinical_data/GSE58689.csv
Gene data columns (first 5): ['GSM1413270', 'GSM1413271', 'GSM1413272', 'GSM1413273', 'GSM1413274']
Clinical data columns (first 5): ['GSM1413270', 'GSM1413271', 'GSM1413272', 'GSM1413273', 'GSM1413274']
Found 38 common samples between gene and clinical data
Initial linked data shape: (38, 3838)
Preview of linked data (first 5 rows, first 5 columns):
            Thyroid_Cancer  Gender    A4GALT       AAA1      AAR2
GSM1413270        

Linked data shape after handling missing values: (10, 3837)
Quartiles for 'Thyroid_Cancer':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1.0
Max: 1.0
The distribution of the feature 'Thyroid_Cancer' in this dataset is severely biased.

Data not usable for the trait study - not saving final linked data.
